# Génération de texte avec les réseaux de neurones

Dans ce notebook, nous allons créer un réseau qui peut générer du texte, ici nous montrons que cela se fait caractère par caractère. Super post à lire ici : http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Nous avons organisé le processus en "étapes" afin que vous puissiez suivre facilement avec vos propres ensembles de données.

## Étape 1 : Les Données

Vous pouvez récupérer n'importe quel texte gratuit ici : https://www.gutenberg.org/

Nous allons choisir toutes les œuvres de Shakespeare (que nous avons déjà téléchargées pour vous), principalement pour deux raisons :

1. C'est un grand corpus de textes, il est généralement recommandé d'avoir au moins une source d'un million de caractères au total pour obtenir une génération de texte réaliste.

2. Il a un style très particulier. Comme les données textuelles utilisent un anglais ancien et sont formatées dans le style d'une pièce de théâtre, il nous apparaîtra très clairement si le modèle est capable de reproduire des résultats similaires.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
text = open("shakespeare.txt", 'r').read()
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Comprendre les caractères uniques

In [ ]:
vocab = sorted(set(text))
print(vocab)
print(len(vocab))

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']
84


## Étape 2 : Traitement du texte

### Vectorisation du texte

Nous savons qu'un réseau de neurones ne peut pas prendre en charge les données brutes des chaînes de caractères, nous devons attribuer des numéros à chaque caractère. Créons deux dictionnaires qui peuvent passer d'un index numérique à un caractère et d'un caractère à un index numérique.

In [ ]:
char_to_ind = {u:i for i, u in enumerate(vocab)}
print({k: char_to_ind[k] for k in list(char_to_ind)[:10]})
print(char_to_ind["!"])
ind_to_char = np.array(vocab)
print(ind_to_char[:5])
ind_to_char[2]

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9}
2
['\n' ' ' '!' '"' '&']


'!'

In [ ]:
# Cela convertit le texte en nombres entiers
encoded_text = np.array([char_to_ind[c] for c in text])
print(f'{text[:200]} => {encoded_text[:200]}')
print()
print(encoded_text.shape)


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  => [ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75
  1 56 74  1 75 63 60  1 73 64 71 60 73  1 74 63 70 76 67 59  1 57 80  1
 75 64 68 60  1 59 60 58 60 56 74 60  8  0  1  1 33 64 74  1 75 60 69 59
 60 73  1 63 60 64 73  1 68 64 62 63 75  1 57 60 56 73  1 63 64 74  1 68
 60 68 70 73 80 21  0  1]

(5445609,)


## Étape 3 : Création de batches

Globalement, ce que nous essayons de faire, c'est de faire en sorte que le modèle prévoie le caractère suivant le plus probable, compte tenu d'une séquence historique de caractères. C'est à nous (l'utilisateur) de choisir la longueur de cette séquence historique. Une séquence trop courte et nous n'aurons pas assez d'informations (par exemple, étant donné la lettre "a", quel est le prochain caractère), une séquence trop longue et l'entraînement prendra trop de temps et risque de sur-entraîner (au risque d'obtenir une séquence de caractères qui ne sont pas pertinents pour des caractères plus éloignés). Bien qu'il n'y ait pas de choix correct de longueur de séquence, vous devez considérer le texte lui-même, la longueur des phrases normales qu'il contient et avoir une idée raisonnable des caractères/mots qui sont pertinents les uns pour les autres.

In [ ]:
def average_chars_per_line(text, start_line, end_line):
    lines = text.split('\n')[start_line-1:end_line]
    total_chars = sum(len(line) for line in lines)
    average_chars = total_chars / len(lines)
    return lines, total_chars, average_chars

lines, total_chars, average_chars = average_chars_per_line(text, 3, 5)
print(f'Le nombre moyen de caractères pour {len(lines)} lignes est {average_chars:.2f} | total de caractères: {total_chars}')
# Nous avons un total de caractères de 131 pour 3 lignes, on peut donc choisir une longueur de séquence de 120 caractères
# pour avoir assez d'informations pour capturer l'essence du texte et pas trop grand pour ne pas être trop lents pendant l'entraînement.

Le nombre moyen de caractères pour 3 lignes est 43.67 | total de caractères: 131


### Séquences d'Entraînement

Les données textuelles réelles seront la séquence de texte décalée d'un caractère vers l'avant.

Par exemple :

*   Sequence In: "Hello my nam"
*   Sequence Out: "ello my name"

Nous pouvons utiliser la fonction `tf.data.Dataset.from_tensor_slices` pour convertir un vecteur de texte en un flux d'indices de caractères.

In [ ]:
seq_len = 120
total_num_seq = len(text) // (seq_len + 1)
print("Nombre total de séquences:", total_num_seq)

Nombre total de séquences: 45005


In [ ]:
# Crée un dataset à partir du texte encodé
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

# Divise le dataset en séquences de longueur seq_len + 1
# drop_remainder=True permet de supprimer les séquences qui ne sont pas de la longueur souhaitée
sequences = char_dataset.batch(seq_len + 1, drop_remainder=True)

Maintenant que nous avons nos séquences, nous allons effectuer les étapes suivantes pour chacune d'entre elles afin de créer nos séquences de texte cible :

1. Saisir la séquence de texte d'entrée
2. Assigner la séquence de texte cible comme séquence de texte d'entrée décalée d'un pas en avant
3. Regroupez-les en un tuple

In [ ]:
def create_seq_targets(seq):
    input_txt = seq[:-1] # Hello my nam
    target_txt = seq[1:] # ello my name
    return input_txt, target_txt # (Hello my nam, ello my name)

In [ ]:
dataset = sequences.map(create_seq_targets)
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Générer des batches d'entraînement

Maintenant que nous avons les séquences réelles, nous allons créer les lots, nous voulons mélanger ces séquences dans un ordre aléatoire, de sorte que le modèle ne s'adapte à aucune section du texte, mais puisse au contraire générer des caractères à partir de n'importe quel texte de départ.

In [ ]:
batch_size = 128 # Nombre de séquences dans chaque batch
buffer_size = 10000 # Nous prenons 10000 séquences pour les mélanger aléatoirement
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset
# ici chaque batch contient 128 séquences, chaque séquence contient 120 caractères et chaque caractère est représenté par un nombre entier

<_BatchDataset element_spec=(TensorSpec(shape=(128, 120), dtype=tf.int64, name=None), TensorSpec(shape=(128, 120), dtype=tf.int64, name=None))>

## Étape 4 : Création du modèle

Nous utiliserons un modèle basé sur le LSTM avec quelques caractéristiques supplémentaires, notamment une couche embedding pour commencer et **deux** couches LSTM. Nous avons basé cette architecture de modèle sur le [DeepMoji](https://deepmoji.mit.edu/) et le code source original peut être trouvé [ici](https://github.com/bfelbo/DeepMoji).

La couche embedding servira de couche d'entrée, qui crée essentiellement une table de consultation qui fait correspondre les indices numériques de chaque caractère à un vecteur avec un nombre de dimensions "embedding dim". Comme vous pouvez l'imaginer, plus cette taille d'embedding est grande, plus l'entraînement est complexe. C'est similaire à l'idée derrière word2vec, où les mots sont mis en correspondance avec un espace à n dimensions. L'embedding avant le feeding directe dans le LSTM conduit généralement à des résultats plus réalistes.

In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [ ]:
# Longueur du vocabulaire en caractères
vocab_size = len(vocab)

# La dimension embedding
embed_dim = 64

# Nombre d'unitées RNN
rnn_neurons = 1026

def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    model.summary()
    return model

In [ ]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 64)           5376      
                                                                 
 gru (GRU)                   (128, None, 1026)         3361176   
                                                                 
 dense (Dense)               (128, None, 84)           86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Étape 5 : Entraînement du modèle

Assurons-nous que tout va bien avec notre modèle avant de passer trop de temps sur l'entraînement ! Passons en lot pour confirmer que le modèle prédit actuellement des caractères aléatoires sans aucun entraînement.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1): # 1 élément du dataset
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "=> (batch_size, sequence_length, vocab_size)")

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # transforme les probabilités en nombres entiers (indices)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # supprime la dimension inutile
print(sampled_indices)
ind_to_char[sampled_indices] # convertit les nombres entiers en caractères

(128, 120, 84) => (batch_size, sequence_length, vocab_size)
[72  3 52 61 25 23 15  4 56 56 53 81 24 23 54 76  5 57 58 50 11 43 40 34
 63 22 58 28 60 23 27 42  5 54 78 14 67 56  2 78 23 11  9 69 57 80 15 15
 27 62 60 50 68 13 19 76 11 51 20 52 82 65 60  1 17 31 65 29 28 77 70  6
 16  2 49 80 41  4 36 29 47 76 66 58 69 67  4 48  3 64  5 42 48 42 12 82
 34  7 63 44 35 60  9 63  9 29 19 19 60 68 25 72 41  6 22 71 72 14 48 66]


array(['q', '"', '[', 'f', '?', '<', '4', '&', 'a', 'a', ']', 'z', '>',
       '<', '_', 'u', "'", 'b', 'c', 'Y', '0', 'R', 'O', 'I', 'h', ';',
       'c', 'C', 'e', '<', 'B', 'Q', "'", '_', 'w', '3', 'l', 'a', '!',
       'w', '<', '0', '-', 'n', 'b', 'y', '4', '4', 'B', 'g', 'e', 'Y',
       'm', '2', '8', 'u', '0', 'Z', '9', '[', '|', 'j', 'e', ' ', '6',
       'F', 'j', 'D', 'C', 'v', 'o', '(', '5', '!', 'X', 'y', 'P', '&',
       'K', 'D', 'V', 'u', 'k', 'c', 'n', 'l', '&', 'W', '"', 'i', "'",
       'Q', 'W', 'Q', '1', '|', 'I', ')', 'h', 'S', 'J', 'e', '-', 'h',
       '-', 'D', '8', '8', 'e', 'm', '?', 'q', 'P', '(', ';', 'p', 'q',
       '3', 'W', 'k'], dtype='<U1')

In [ ]:
epochs = 30
model.fit(dataset, epochs=epochs)

Epoch 1/30
351/351 [==============================] - 50s 125ms/step - loss: 2.5675
Epoch 2/30
351/351 [==============================] - 45s 123ms/step - loss: 1.7607
Epoch 3/30
351/351 [==============================] - 45s 123ms/step - loss: 1.4809
Epoch 4/30
351/351 [==============================] - 46s 123ms/step - loss: 1.3541
Epoch 5/30
351/351 [==============================] - 46s 127ms/step - loss: 1.2879
Epoch 6/30
351/351 [==============================] - 48s 131ms/step - loss: 1.2453
Epoch 7/30
351/351 [==============================] - 47s 126ms/step - loss: 1.2144
Epoch 8/30
351/351 [==============================] - 47s 130ms/step - loss: 1.1901
Epoch 9/30
351/351 [==============================] - 48s 133ms/step - loss: 1.1695
Epoch 10/30
351/351 [==============================] - 51s 135ms/step - loss: 1.1517
Epoch 11/30
351/351 [==============================] - 47s 127ms/step - loss: 1.1364
Epoch 12/30
351/351 [==============================] - 48s 131ms/step - lo

In [ ]:
from tensorflow.keras.models import load_model
model.save("model.keras")

## Étape 6 : Génération du texte

Actuellement, notre modèle ne prévoit que 128 séquences à la fois. Nous pouvons créer un nouveau modèle qui n'attend qu'un batch_size=1. Nous pouvons créer un nouveau modèle avec cette taille de batch, puis charger les poids de nos modèles sauvegardés. Ensuite, appelez .build() sur le modèle :

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1) # batch_size=1 pour que le modèle puisse accepter une séquence de n'importe quelle longueur
model.load_weights("model.keras") # charger les poids du modèle entraîné
model.build(tf.TensorShape([1, None])) # construire le modèle avec une longueur de séquence de 1
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 64)             5376      
                                                                 
 gru_1 (GRU)                 (1, None, 1026)           3361176   
                                                                 
 dense_1 (Dense)             (1, None, 84)             86268     
                                                                 
Total params: 3452820 (13.17 MB)
Trainable params: 3452820 (13.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:

"""
Génère du texte en utilisant un modèle donné et une chaîne de départ.

Paramètres:
- model: Le modèle entraîné utilisé pour la génération de texte.
- start_string: La chaîne initiale pour démarrer le processus de génération.
- num_generate: Le nombre de caractères à générer (par défaut: 1000).
- temperature: La valeur de température pour contrôler l'aléatoire du texte généré (par défaut: 1.0).

Retourne:
- Le texte généré en tant que chaîne.
"""
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    # Convertit la chaîne de départ en nombres (vectorisation)
    input_eval = [char_to_ind[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Chaîne vide pour stocker le texte généré
    generated_text = []

    # Abaisse la température pour rendre la sortie plus aléatoire
    model.reset_states() # prendre en compte le taille de batch_size à 1
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) # retirer la dimension faite par expand_dims

        # Apply temperature to the predictions
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Passe le caractère prédit comme prochaine entrée du modèle
        input_eval = tf.expand_dims([predicted_id], 0)
        generated_text.append(ind_to_char[predicted_id])

    return start_string + ''.join(generated_text)

In [ ]:
print(generate_text(model=model, start_string="Romeo", num_generate=500))

Romeove, I am s
  PETOR. Most gracious Prince! think you all 'tis a fair return.
    Yet, it grows a that thus have sport alive,
    And say 'The man I'll lay their vows, haply, my lord;
    The cleare is to this paper for the girn
    Had not it for thy print you come, and I will pay.
    How cam'd your counsel when I am a cut,
    It was my reapon, I
    Hast discovered horns, a practice for
    A prayer and twenty years and bad a breach:
    I answer in my title to the heavens
    Give to my wealt
